In [1]:
import pandas as pd
import warnings 
import datetime

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("2019 all data.csv")

In [3]:
df = df[(df.position_name=="Продавец-кассир") & (df.is_active_action==True) & (df.id_log!=-1)]

In [4]:
# челики со сменой вакансий 
serias_id_parther_vacancy = df.id_partner[df.stage_name=="Смена вакансии"]
df_vacancy = df[df.id_partner.isin(serias_id_parther_vacancy)]
df = df[~df.id_partner.isin(serias_id_parther_vacancy)]

In [5]:
# принятых кандидатов 
serias_id_parther_exist = df.id_partner[df.result_name=="Принят"]
df = df[df.id_partner.isin(serias_id_parther_exist)]


In [6]:
len(df.id_partner.unique())

59390

In [7]:
df = df.sort_values(by=['id_partner', 'id_applicant', 'action_date'])

In [7]:
# df.stage_name.unique()

array(['Обработка', 'Интервью с руководителем', 'СБ', 'Оформление',
       'Назначение времени на оформление/медосмотр', 'Отказался',
       'Не прошел СБ', 'Ожидание', 'Выход на работу', 'Резерв',
       'Черный список', 'Мы отказали',
       'Групповое собеседование и тестирование', 'Гайд-интервью'],
      dtype=object)

In [8]:
partners = df.id_partner.unique()
steps = set(["Обработка", "Интервью с руководителем", "СБ", "Оформление", "Назначение времени на оформление/медосмотр"])
good_partners = [] # кандидаты со всеми этапами 
bad_partners = []  # кандидаты с некоторыми этапами 
for partner in partners:
    df_p = df[df.id_partner==partner]
    if set(df_p.stage_name).issubset(steps):
        good_partners.append(df_p.id_partner.iloc[0])
    else:
        bad_partners.append(df_p.id_partner.iloc[0])

print('good = ', len(good_partners), ' bad = ', len(bad_partners))

good =  55077  bad =  4313


In [9]:
df_good = df[df.id_partner.isin(good_partners)]
# df_bad = df[df.id_partner.isin(bad_partners)]

In [9]:
# series_id_parther_vacancy_exist = df_vacancy.id_partner[df_vacancy.result_name=="Принят"]
# df_vacancy = df_vacancy[df_vacancy.id_partner.isin(series_id_parther_vacancy_exist)]
# df_vacancy.to_excel("vacancy.xlsx", index=False)

In [12]:
df_good.to_csv('good_partner.csv', index=False)

In [ ]:
# df_bad.to_excel("bad.xlsx", index=False)

In [13]:
df_tth = pd.DataFrame([], columns=['id_partner', 'tth'])

In [14]:
for partner in good_partners:
    df_p = df_good[df_good.id_partner==partner]
    df_p.action_date = pd.to_datetime(df_p['action_date'], format='%Y-%m-%d %H:%M:%S.%f%z')
    raw_good = df_p[df_p.result_name=="Принят"]
    date, id_applicant_ = raw_good.action_date.iloc[0], raw_good.id_applicant.iloc[0]
    df_p = df_p[df_p.id_applicant==id_applicant_]
    raw = {'id_applicant': id_applicant_ ,'id_partner': partner, 'tth':date-df_p.action_date.min()}
    df_tth = df_tth.append(raw, ignore_index=True)
                     
df_tth

,id_partner,tth,id_applicant
0,531634,7 days 01:28:41.275086,4089877.0
1,531684,3 days 00:56:53.636105,3791739.0
2,532567,7 days 20:49:23.334812,3452857.0
3,532650,12 days 00:48:13.590964,3339449.0
4,533824,13 days 23:00:52.102205,4140195.0
...,...,...,...
55072,3888013,14 days 03:18:05.971615,3535567.0
55073,3891266,2 days 02:55:55.482857,3334936.0
55074,3892298,13 days 16:56:38.099569,3441230.0
55075,3896441,7 days 19:02:54.123351,3337215.0


In [15]:
df_tth.describe()

,tth,id_applicant
count,55077,5.507700e+04
mean,12 days 11:43:07.593133101,3.735607e+06
std,15 days 00:00:34.011292347,2.404820e+05
min,0 days 00:00:00,3.330795e+06
25%,5 days 04:34:54.162873,3.522303e+06
50%,8 days 02:27:21.451325,3.745066e+06
75%,14 days 17:52:04.929912,3.936683e+06
max,266 days 18:55:30.748658,4.184896e+06


In [16]:
df_tth.to_excel('good_tth_v3.xlsx')

In [20]:
good_applicant_big_tth = pd.read_excel('good_tth_v3.xlsx', sheet_name='applicant')
l = good_applicant_big_tth.id_applicant.values.tolist()

In [21]:
df_good_big_thh = df_good[df_good.id_applicant.isin(l)]
df_good_big_thh

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_active_action,is_open_action
3147249,4089877,531634,22169026,47429,2019-09-19 10:03:07.871391+00,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,True,False
3147250,4089877,531634,22169027,47429,2019-09-19 10:03:08.431434+00,Обработка,Проверка формы обращения,Не резерв,Продавец-кассир,True,False
3147248,4089877,531634,22169041,47429,2019-09-19 10:03:20.237747+00,Интервью с руководителем,Назначить интервью с руководителем,Встреча назначена,Продавец-кассир,True,False
3147247,4089877,531634,22220816,47429,2019-09-20 16:19:51.725286+00,Интервью с руководителем,Узнать результаты интервью с руководителем,Недозвон,Продавец-кассир,True,False
3147246,4089877,531634,22278256,47429,2019-09-24 13:11:08.84847+00,Интервью с руководителем,Узнать результаты интервью с руководителем,Недозвон,Продавец-кассир,True,False
...,...,...,...,...,...,...,...,...,...,...,...
3208799,3752441,3896610,19725530,9583,2019-05-27 09:35:20.037281+00,Оформление,Назначить время на оформление/медосмотр,Время назначено,Продавец-кассир,True,False
3208798,3752441,3896610,19787731,9583,2019-05-29 11:18:56.577667+00,Оформление,Узнать результаты приема,В ожидании,Продавец-кассир,True,False
3208797,3752441,3896610,19850588,9583,2019-05-31 15:18:58.823646+00,Оформление,Узнать результаты приема,В ожидании,Продавец-кассир,True,False
3208796,3752441,3896610,19851280,9583,2019-05-31 15:35:35.788558+00,Оформление,Узнать результаты приема,В ожидании,Продавец-кассир,True,False


In [23]:
# df_good_big_thh.to_csv('good_tth_applicant.csv', index=False)

In [142]:
# РАСЧЕТ МЕТРИК 
# tth_metrics = pd.read_excel('good_tth_applicant.xlsx', sheet_name='Sheet1')
# id_applicant_ = tth_metrics.id_applicant.values.tolist()
# df_metrics = df_good[df_good.id_applicant.isin(id_applicant_)]
# df_metrics

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_active_action,is_open_action
4449352,4076843,539640,22077478,34582,2019-09-16 10:25:38.162287+00,Обработка,Провести первичную обработку,Подходит,Продавец-кассир,True,False
4449351,4076843,539640,22077480,34582,2019-09-16 10:25:38.811886+00,Обработка,Проверка формы обращения,Не резерв,Продавец-кассир,True,False
4449350,4076843,539640,22077482,34582,2019-09-16 10:25:39.525034+00,Интервью с руководителем,Назначить интервью с руководителем,Встреча назначена,Продавец-кассир,True,False
4449349,4076843,539640,22077484,34582,2019-09-16 10:25:40.098674+00,Интервью с руководителем,Узнать результаты интервью с руководителем,Подходит,Продавец-кассир,True,False
4449348,4076843,539640,22077486,34582,2019-09-16 10:25:40.755303+00,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,True,False
...,...,...,...,...,...,...,...,...,...,...,...
2975782,3535567,3888013,18313323,32971,2019-03-14 09:34:50.770932+00,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,True,False
2975780,3535567,3888013,18326492,32971,2019-03-14 13:16:49.621594+00,СБ,Узнать результаты СБ,Подходит,Продавец-кассир,True,False
2975781,3535567,3888013,18326493,32971,2019-03-14 13:16:50.248863+00,СБ,Проверка формы обращения,Не резерв,Продавец-кассир,True,False
2975776,3535567,3888013,18629708,32971,2019-03-28 12:52:49.218771+00,Оформление,Назначить время на оформление/медосмотр,Время назначено,Продавец-кассир,True,False


In [31]:
df_metrics = df_good_big_thh.copy()
id_applicant_ = df_metrics.id_applicant.unique()
len(id_applicant_)

53454

In [32]:

df_result = pd.DataFrame([], columns=['id_applicant', 'waiting', 'not call', 'count of call', 'between SB'])
# id_applicant_2 = [4131077]
for applicant in id_applicant_:
    df_app = df_metrics[df_metrics.id_applicant==applicant]
    df_app.reset_index(inplace=True)
    df_app.action_date = pd.to_datetime(df_app['action_date'], format='%Y-%m-%d %H:%M:%S.%f%z')
    waiting = df_app[df_app.result_name=="В ожидании"].action_date.max() - df_app[df_app.result_name=="В ожидании"].action_date.min()
    not_call_sum = datetime.timedelta(days=0, seconds=0, minutes=0, hours=0)
    for index, row in df_app.iterrows():   
        if index < len(df_app):
            if row.result_name == "Недозвон" and df_app.iloc[index+1].result_name == "Недозвон":
                not_call = df_app.iloc[index+1].action_date - df_app.iloc[index].action_date
                not_call_sum += not_call
    count_of_call = len(df_app[df_app.result_name=="Недозвон"])
    try:
        between_SB = df_app[(df_app.stage_name=='СБ') & (df_app.result_name=='Подходит')].action_date.iloc[0] - df_app[df_app.activity_name=='Подготовить анкету кандидата'].action_date.iloc[0]
    except:
        print(applicant)
        between_SB=0
#     print(between_SB)
    df_result = df_result.append({'id_applicant': applicant, 'waiting': 0 if pd.isnull(waiting) else waiting, 'not call': not_call_sum, 'count of call': count_of_call, 'between SB': between_SB}, ignore_index=True)

3388945
3393687
3392556
3389411
3386663
3350559
3393480
3385225
3391443
3389006
3389352
3386637
3384513
3390792
3381983
3390427
3391016
3387758
3385189
3389993
3373425
3385011
3385548
3393489
3381972
3392197
3385322
3390585
3393785
3340872
3384009
3391382
3389370
3393099
3368949
3391123
3386755
3383897
3384575
3384502
3386228
3386529
3391568
3392711
3389713
3392431
3393943
3391012
3390791
3387851
3386904
3391191
3388177
3391279
3389246
3386579
3384001
3388508
3391411
3385209
3385293
3391254
3390731
3392733
3388125
3389243
3364059
3365614
3388013
3390305
3391118
3382906
3390638
3383852
3392998
3390986
3384398
3375853
3393819
3385267
3392171
3374469
3383745
3390347
3379252
3387688
3389075
3385500
3393179
3389332
3387690
3349973
3388363
3390783
3390411
3388861
3385552
3388491
3389435
3371545
3368765
3387438
3377355
3380327
3385512
3368641
3390924
3388838
3386661
3385132
3372593
3393618
3390051
3371991
3347428
3387647
3382296
3384170
3353056
3386391
3370356
3354969
3386567
3391857
3358344


In [33]:
df_result

,id_applicant,waiting,not call,count of call,between SB
0,4089877,0,4 days 20:37:22.294147,3,0 days 00:00:00.508406
1,3791739,0,0 days 00:00:00,1,0 days 18:27:03.311687
2,3452857,0 days 00:00:00,0 days 00:00:00,0,0 days 02:24:16.367776
3,3339449,0,0 days 00:00:00,0,0 days 03:30:41.528014
4,4140195,0,0 days 00:00:00,3,0 days 19:53:55.300702
...,...,...,...,...,...
53449,3535567,0,0 days 00:00:00,0,0 days 03:41:58.850662
53450,3334936,0 days 00:00:00,0 days 00:00:00,1,0 days 02:53:01.314204
53451,3441230,0,0 days 20:05:06.897319,2,0 days 17:27:42.810607
53452,3337215,0,0 days 00:00:00,0,0 days 18:37:28.178891


In [145]:
# df_good[df_good.id_applicant==3385225]

In [36]:
df_tth_for_merge = pd.read_excel('good_tth_v3.xlsx', sheet_name='+ln tth', names=['tth', 'id_applicant', 'ln tth'])
df_end = df_result.merge(df_tth_for_merge, on='id_applicant')
df_end

,id_applicant,waiting,not call,count of call,between SB,tth,ln tth
0,4089877,0,4 days 20:37:22.294147,3,0 days 00:00:00.508406,7.061589,1.954670
1,3791739,0,0 days 00:00:00,1,0 days 18:27:03.311687,3.039510,1.111696
2,3452857,0 days 00:00:00,0 days 00:00:00,0,0 days 02:24:16.367776,7.867631,2.062757
3,3339449,0,0 days 00:00:00,0,0 days 03:30:41.528014,12.033491,2.487694
4,4140195,0,0 days 00:00:00,3,0 days 19:53:55.300702,13.958936,2.636120
...,...,...,...,...,...,...,...
53449,3535567,0,0 days 00:00:00,0,0 days 03:41:58.850662,14.137569,2.648836
53450,3334936,0 days 00:00:00,0 days 00:00:00,1,0 days 02:53:01.314204,2.122170,0.752439
53451,3441230,0,0 days 20:05:06.897319,2,0 days 17:27:42.810607,13.705997,2.617833
53452,3337215,0,0 days 00:00:00,0,0 days 18:37:28.178891,7.793682,2.053313


In [37]:
df_end.to_excel('metrics.xlsx', index=False)

In [17]:
df_tth_small = df_tth[df_tth.tth < datetime.timedelta(days=1)]
df_tth_small['tth'] = df_tth_small['tth'].apply(lambda x: x.total_seconds())

In [18]:
df_tth_small.dtypes

id_partner       object
tth             float64
id_applicant    float64
dtype: object

In [19]:
df_tth_small.to_excel('small_tth.xlsx')

In [ ]:
167